# 시작

In [1]:
import os
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

from pycaret.classification import *
from pycaret.regression import *

import warnings

# 경고 메시지 숨기기
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 50) 
pd.set_option('display.max_rows', 1000) 

from matplotlib import font_manager
from matplotlib import rc
# 'Nanum Gothic' 폰트 설정
# font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# font_prop = font_manager.FontProperties(fname=font_path)
# rc('font', family=font_prop.get_name())

# matplotlib의 기본 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'

In [2]:
df = pd.read_csv('250114_ACC_date_0101_0114_ver1.csv') ; df

,ID,ADWORD_ID,ADWORD_TYPE,CAMPAIGN_ID,ADVERTISER,DESCRIPTION,DATE,TIME,SPEND,DB_COUNT,CLICKS,IMPRESSIONS,REVENUE,SALES,ACT_DAYS,ADWORD_SET_ID
0,706756228442-2,706756228442,3,21491987475,7.0,대구점_인모드_페이스북,2025-01-10,14:45,2,0,0,1,-2.0,0.0,1,164366866799
1,706756228442-2,706756228442,3,21491987475,7.0,대구점_인모드_페이스북,2025-01-10,15:00,2,0,0,1,-2.0,0.0,1,164366866799
2,706756228442-2,706756228442,3,21491987475,7.0,대구점_인모드_페이스북,2025-01-10,15:15,2,0,0,1,-2.0,0.0,1,164366866799
3,706756228442-2,706756228442,3,21491987475,7.0,대구점_인모드_페이스북,2025-01-10,15:30,2,0,0,1,-2.0,0.0,1,164366866799
4,706756228442-2,706756228442,3,21491987475,7.0,대구점_인모드_페이스북,2025-01-10,15:45,2,0,0,1,-2.0,0.0,1,164366866799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467359,728844863034-1,728844863034,3,22114323339,6.0,대전점_무제한_29만원_가격알아보기_검정mkt,2025-01-14,12:15,12,0,0,4,-12.0,0.0,1,173037196309
1467360,728844863034-1,728844863034,3,22114323339,6.0,대전점_무제한_29만원_가격알아보기_검정mkt,2025-01-14,12:30,12,0,0,4,-12.0,0.0,1,173037196309
1467361,728844863034-1,728844863034,3,22114323339,6.0,대전점_무제한_29만원_가격알아보기_검정mkt,2025-01-14,12:45,12,0,0,4,-12.0,0.0,1,173037196309
1467362,728844863034-1,728844863034,3,22114323339,6.0,대전점_무제한_29만원_가격알아보기_검정mkt,2025-01-14,13:00,12,0,0,4,-12.0,0.0,1,173037196309


In [ ]:
# # CSV 파일 경로 설정
# file1 = '240826_date_0720_0825_ver1.csv'
# file2 = '240906_date_0826_0906_ver1.csv'

# # CSV 파일 불러오기
# df1 = pd.read_csv(file1)
# df2 = pd.read_csv(file2)

# # 위아래로 합치기
# df = pd.concat([df1, df2], ignore_index=True)

# # 결과 확인
# print(df)

### 공통 전처리

In [3]:
df = df.sort_values(by=['ADWORD_ID', 'DATE', 'TIME']).reset_index(drop=True)

# DATE 열을 datetime 형식으로 변환
df['DATE'] = pd.to_datetime(df['DATE'])

# df = df.sort_values(by=['ADWORD_ID', 'DATE'])

# # 각 ADWORD_ID 그룹에 대해 시작일부터 현재 날짜까지의 일수를 계산합니다.
# df['ACT_DAYS'] = df.groupby('ADWORD_ID')['DATE'].transform(lambda x: (x - x.min()).dt.days + 1)


# TIME 열을 문자열로 변환 후 시간 형식으로 처리
df['TIME'] = df['TIME'].astype(str).str.slice(0, 5)
df['TIME'] = pd.to_datetime(df['TIME'], format='%H:%M').dt.time

# TIME을 분 단위로 변환
df['TIME_MINUTES'] = df['TIME'].apply(lambda x: x.hour * 60 + x.minute)

df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month
df['DAY'] = df['DATE'].dt.day
df['WEEK'] = df['DATE'].dt.dayofweek

In [6]:
df['ADVERTISER'] = df['ADVERTISER'].astype('category')

In [7]:
df['ADWORD_ID'] = df['ADWORD_ID'].astype('category')

# 노출 테스트

In [ ]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split
import gc

# -----------------------------
# 0. 원본 DataFrame(df)에서 테스트 데이터 생성
#    DATE 컬럼이 '2025-01-01' ~ '2025-01-14' 사이인 행만 선택
# -----------------------------
df_test = df[df['DATE'].between('2025-01-01', '2025-01-14')].copy()
df_test.reset_index(drop=True, inplace=True)

# -----------------------------
# 1. 결과 저장 관련 변수 설정
# -----------------------------
header = ["ADWORD_ID", "DATE", "TIME", "ACT_DAYS", "IMPRESSIONS"]
# 각 shift_interval에 대해, 오른쪽에 예측, 실제, 점수 컬럼 추가
shift_intervals = [1440, 2880, 4320, 5760, 7200, 8640, 10080]  # 분 단위
for si in shift_intervals:
    header.extend([f"IMPRESSIONS_{si}M_LATER", f"IMPRESSIONS_{si}M_ACTUAL", f"Score_{si}M_LATER"])

results = []            # 결과 저장 리스트
file_count = 1          # 파일 번호
chunk_size = 1_000_000  # 한 파일당 최대 행 수

# -----------------------------
# 2. 테스트 날짜 범위 설정 (2025-01-01 ~ 2025-01-14)
#    (이미 df_test가 위에서 생성됨)
# -----------------------------
dates = pd.date_range(start='2025-01-01', end='2025-01-14')

# -----------------------------
# 3. 예측된 모델 결과(여러 shift_interval별) 오른쪽에 컬럼으로 추가하기
# -----------------------------
# df_result는 테스트 데이터 df_test를 기반으로 합니다.
df_result = df_test.reset_index(drop=True)

for si in shift_intervals:
    shift_value = si // 15  # 15분 단위

    # (A) 타깃 컬럼: IMPRESSIONS를 shift하여 'IMPRESSIONS_H_LATER' 생성
    df_result['IMPRESSIONS_H_LATER'] = (
        df_result.groupby('ADWORD_ID')['IMPRESSIONS']
                 .transform(lambda x: x.shift(-shift_value))
                 .fillna(0)
    )

    # (B) 마지막 구간(shift_value개)에 대해 보정:
    def fill_last_n_with_last_value(group):
        if len(group) >= shift_value:
            fill_value = group.iloc[-1]['IMPRESSIONS']  # 그룹의 마지막 IMPRESSIONS 값
            group.iloc[-shift_value:, group.columns.get_loc('IMPRESSIONS_H_LATER')] = fill_value
        return group

    df_result = df_result.groupby('ADWORD_ID').apply(fill_last_n_with_last_value).reset_index(drop=True)

    # (C) 피처 및 타깃 정의  
    #     피처: 기타 컬럼들을 사용, 타깃: 'IMPRESSIONS_H_LATER'
    X = df_result[['ADWORD_ID', 'ACT_DAYS', 'TIME_MINUTES', 'MONTH', 'WEEK',
                   'ADVERTISER', 
                   'CLICKS', 'SPEND', 'DB_COUNT', 'REVENUE','SALES']]
    y_actual = df_result['IMPRESSIONS_H_LATER']
    
    # 모델 파일 불러오기 (모델 파일명도 IMPRESSIONS 예측에 맞게 변경)
    model_filename = f'250102_IMP_{shift_interval}M_VER1.pkl'
    load_rf = joblib.load(model_filename)
    y_pred = load_rf.predict(X)
    
    # 예측 결과를 오른쪽에 컬럼으로 추가
    df_pred = pd.DataFrame(y_pred, columns=[f'IMPRESSIONS_{si}M_LATER'])
    df_result = pd.concat([df_result, df_pred], axis=1)
    
    # 실제 미래 IMPRESSIONS 값: 그룹별 shift(-shift_value)를 적용하여 생성
    future_imp_column = f'IMPRESSIONS_{si}M_ACTUAL'
    df_result[future_imp_column] = df_result.groupby('ADWORD_ID')['IMPRESSIONS'].shift(-shift_value)
    
    # (D) Score 계산 함수: 실제와 예측값 비교
    def calculate_score(y, y_pred):
        if pd.isnull(y) or pd.isnull(y_pred):
            return None
        y_rounded = round(y)
        y_pred_rounded = round(y_pred)
        if y_rounded != 0:
            error_rate = abs((y_rounded - y_pred_rounded) / y_rounded) * 100
            return abs(y_rounded / (y_rounded - y_pred_rounded)) * 100 if error_rate > 100 else 100 - error_rate
        return None

    score_column = f"Score_{si}M_LATER"
    df_result[score_column] = [
        calculate_score(actual, pred)
        for actual, pred in zip(df_result[future_imp_column], df_result[f'IMPRESSIONS_{si}M_LATER'])
    ]
    
    overall_score = df_result[score_column].mean()
    print(f"Shift Interval {si}M: Overall Average Score: {overall_score:.2f}")

# -----------------------------
# 4. 최종 컬럼 순서 재정렬 및 Excel로 저장
# -----------------------------
df_result = df_result[header]

file_index = 1
for i in range(0, len(df_result), chunk_size):
    chunk_filename = f"250220.test_results_part1_{file_index}.xlsx"
    df_result.iloc[i:i+chunk_size].to_excel(chunk_filename, index=False)
    print(f"파일 저장 완료: {chunk_filename}")
    file_index += 1

gc.collect()


# 클릭 테스트

In [ ]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split
import gc

# -----------------------------
# 0. 원본 DataFrame(df)에서 테스트 데이터 생성
#    DATE 컬럼이 '2025-01-01' ~ '2025-01-14' 사이인 행만 선택
# -----------------------------
df_test = df[df['DATE'].between('2025-01-01', '2025-01-14')].copy()
df_test.reset_index(drop=True, inplace=True)

# -----------------------------
# 1. 결과 저장 관련 변수 설정
# -----------------------------
header = ["ADWORD_ID", "DATE", "TIME", "ACT_DAYS", "CLICKS"]
# 각 shift_interval에 대해, 오른쪽에 예측, 실제, 점수 컬럼 추가
shift_intervals = [1440, 2880, 4320, 5760, 7200, 8640, 10080]  # 분 단위
for si in shift_intervals:
    header.extend([f"CLICKS_{si}M_LATER", f"CLICKS_{si}M_ACTUAL", f"Score_{si}M_LATER"])

results = []            # 결과 저장 리스트
file_count = 1          # 파일 번호
chunk_size = 1_000_000  # 한 파일당 최대 행 수

# -----------------------------
# 2. 테스트 날짜 범위 설정 (2025-01-01 ~ 2025-01-14)
#    (이미 df_test가 위에서 생성됨)
# -----------------------------
dates = pd.date_range(start='2025-01-01', end='2025-01-14')

# -----------------------------
# 3. 예측된 모델 결과(여러 shift_interval별) 오른쪽에 컬럼으로 추가하기
# -----------------------------
# df_result는 테스트 데이터 df_test를 기반으로 합니다.
df_result = df_test.reset_index(drop=True)

for si in shift_intervals:
    shift_value = si // 15  # 15분 단위

    # (A) 타깃 컬럼: CLICKS를 shift하여 'CLICKS_H_LATER' 생성
    df_result['CLICKS_H_LATER'] = (
        df_result.groupby('ADWORD_ID')['CLICKS']
                 .transform(lambda x: x.shift(-shift_value))
                 .fillna(0)
    )

    # (B) 마지막 구간(shift_value개)에 대해 보정:
    def fill_last_n_with_last_value(group):
        if len(group) >= shift_value:
            fill_value = group.iloc[-1]['CLICKS']  # 그룹의 마지막 CLICKS 값
            group.iloc[-shift_value:, group.columns.get_loc('CLICKS_H_LATER')] = fill_value
        return group

    df_result = df_result.groupby('ADWORD_ID').apply(fill_last_n_with_last_value).reset_index(drop=True)

    # (C) 피처 및 타깃 정의  
    #     피처: 기타 컬럼들을 사용, 타깃: 'CLICKS_H_LATER'
    X = df_result[['ADWORD_ID', 'ACT_DAYS', 'TIME_MINUTES', 'MONTH', 'WEEK',
                   'ADVERTISER', 
                   'IMPRESSIONS', 'SPEND', 'DB_COUNT', 'REVENUE','SALES']]
    y_actual = df_result['CLICKS_H_LATER']
    
    # 모델 파일 불러오기 (모델 파일명도 CLICKS 예측에 맞게 변경)
    model_filename = f'250102_CLK_{shift_interval}M_VER1.pkl'
    load_rf = joblib.load(model_filename)
    y_pred = load_rf.predict(X)
    
    # 예측 결과를 오른쪽에 컬럼으로 추가
    df_pred = pd.DataFrame(y_pred, columns=[f'CLICKS_{si}M_LATER'])
    df_result = pd.concat([df_result, df_pred], axis=1)
    
    # 실제 미래 CLICKS 값: 그룹별 shift(-shift_value)를 적용하여 생성
    future_clicks_column = f'CLICKS_{si}M_ACTUAL'
    df_result[future_clicks_column] = df_result.groupby('ADWORD_ID')['CLICKS'].shift(-shift_value)
    
    # (D) Score 계산 함수: 실제와 예측값 비교
    def calculate_score(y, y_pred):
        if pd.isnull(y) or pd.isnull(y_pred):
            return None
        y_rounded = round(y)
        y_pred_rounded = round(y_pred)
        if y_rounded != 0:
            error_rate = abs((y_rounded - y_pred_rounded) / y_rounded) * 100
            return abs(y_rounded / (y_rounded - y_pred_rounded)) * 100 if error_rate > 100 else 100 - error_rate
        return None

    score_column = f"Score_{si}M_LATER"
    df_result[score_column] = [
        calculate_score(actual, pred)
        for actual, pred in zip(df_result[future_clicks_column], df_result[f'CLICKS_{si}M_LATER'])
    ]
    
    overall_score = df_result[score_column].mean()
    print(f"Shift Interval {si}M: Overall Average Score: {overall_score:.2f}")

# -----------------------------
# 4. 최종 컬럼 순서 재정렬 및 Excel로 저장
# -----------------------------
df_result = df_result[header]

file_index = 1
for i in range(0, len(df_result), chunk_size):
    chunk_filename = f"250220.test_results_part2_{file_index}.xlsx"
    df_result.iloc[i:i+chunk_size].to_excel(chunk_filename, index=False)
    print(f"파일 저장 완료: {chunk_filename}")
    file_index += 1

gc.collect()


# 지출 테스트

In [9]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split
import gc

# -----------------------------
# 0. 원본 DataFrame(df)에서 테스트 데이터 생성
#    DATE 컬럼이 '2025-01-01' ~ '2025-01-14' 사이인 행만 선택
# -----------------------------
df_test = df[df['DATE'].between('2025-01-01', '2025-01-14')].copy()
df_test.reset_index(drop=True, inplace=True)

# -----------------------------
# 1. 결과 저장 관련 변수 설정
# -----------------------------
header = ["ADWORD_ID", "DATE", "TIME", "ACT_DAYS", "SPEND"]
# 각 shift_interval에 대해, 오른쪽에 예측, 실제, 점수 컬럼 추가
shift_intervals = [1440, 2880, 4320, 5760, 7200, 8640, 10080]  # 분 단위
for si in shift_intervals:
    header.extend([f"SPEND_{si}M_LATER", f"SPEND_{si}M_ACTUAL", f"Score_{si}M_LATER"])

results = []            # 결과 저장 리스트
file_count = 1          # 파일 번호
chunk_size = 1_000_000  # 한 파일당 최대 행 수

# -----------------------------
# 2. 테스트 날짜 범위 설정 (2025-01-01 ~ 2025-01-14)
#    (이미 df_test가 위에서 생성됨)
# -----------------------------
dates = pd.date_range(start='2025-01-01', end='2025-01-14')

# -----------------------------
# 3. 예측된 모델 결과(여러 shift_interval별) 오른쪽에 컬럼으로 추가하기
# -----------------------------
# df_result는 테스트 데이터 df_test를 기반으로 합니다.
df_result = df_test.reset_index(drop=True)

for si in shift_intervals:
    shift_value = si // 15  # 15분 단위

    # (A) 타깃 컬럼: SPEND_diff를 shift하여 'SPEND_diff_H_LATER' 생성
    df_result['SPEND_H_LATER'] = (
        df_result.groupby('ADWORD_ID')['SPEND']
                 .transform(lambda x: x.shift(-shift_value))
                 .fillna(0)
    )

    # (B) 마지막 구간(shift_value개)에 대해 보정:
    def fill_last_n_with_last_value(group):
        if len(group) >= shift_value:
            fill_value = group.iloc[-1]['SPEND']  # 그룹의 마지막 SPEND_diff 값
            group.iloc[-shift_value:, group.columns.get_loc('SPEND_H_LATER')] = fill_value
        return group

    df_result = df_result.groupby('ADWORD_ID').apply(fill_last_n_with_last_value).reset_index(drop=True)

    # (C) 피처 및 타깃 정의  
    #     피처: *_diff 컬럼들을 사용, 타깃: 'SPEND_diff_H_LATER'
    X = df_result[['ADWORD_ID', 'ACT_DAYS', 'TIME_MINUTES', 'MONTH', 'WEEK',
                   'ADVERTISER', 
                   'IMPRESSIONS', 'CLICKS', 'DB_COUNT', 'REVENUE', 'SALES']]
    y_actual = df_result['SPEND_H_LATER']
    
    # 모델 파일 불러오기
    model_filename = f'250102_SPD_{shift_interval}M_VER1.pkl'
    load_rf = joblib.load(model_filename)
    y_pred = load_rf.predict(X)
    
    # 예측 결과를 오른쪽에 컬럼으로 추가
    df_pred = pd.DataFrame(y_pred, columns=[f'SPEND_{si}M_LATER'])
    df_result = pd.concat([df_result, df_pred], axis=1)
    
    # 실제 미래 SPEND_diff 값: 그룹별 shift(-shift_value)를 적용하여 생성
    future_spend_column = f'SPEND_{si}M_ACTUAL'
    df_result[future_spend_column] = df_result.groupby('ADWORD_ID')['SPEND'].shift(-shift_value)
    
    # (D) Score 계산 함수: 실제와 예측값 비교
    def calculate_score(y, y_pred):
        if pd.isnull(y) or pd.isnull(y_pred):
            return None
        y_rounded = round(y)
        y_pred_rounded = round(y_pred)
        if y_rounded != 0:
            error_rate = abs((y_rounded - y_pred_rounded) / y_rounded) * 100
            return abs(y_rounded / (y_rounded - y_pred_rounded)) * 100 if error_rate > 100 else 100 - error_rate
        return None

    score_column = f"Score_{si}M_LATER"
    df_result[score_column] = [
        calculate_score(actual, pred)
        for actual, pred in zip(df_result[future_spend_column], df_result[f'SPEND_{si}M_LATER'])
    ]
    
    overall_score = df_result[score_column].mean()
    print(f"Shift Interval {si}M: Overall Average Score: {overall_score:.2f}")

# -----------------------------
# 4. 최종 컬럼 순서 재정렬 및 Excel로 저장
# -----------------------------
df_result = df_result[header]

file_index = 1
for i in range(0, len(df_result), chunk_size):
    chunk_filename = f"250220.test_results_part2_{file_index}.xlsx"
    df_result.iloc[i:i+chunk_size].to_excel(chunk_filename, index=False)
    print(f"파일 저장 완료: {chunk_filename}")
    file_index += 1

gc.collect()


Shift Interval 60M: Overall Average Score: 51.86
Shift Interval 120M: Overall Average Score: 49.50
Shift Interval 780M: Overall Average Score: 44.18
Shift Interval 840M: Overall Average Score: 44.07
파일 저장 완료: 250220.test_results_part2_1.xlsx
파일 저장 완료: 250220.test_results_part2_2.xlsx


8829938